In [1]:
import json
import pandas as pd
from collections import Counter
import numpy as np

# Charge les transformations détectées
with open('../data/detected_transformations.json') as f:
    transformations = json.load(f)

# Vérifie la structure des données
print("Type de l'objet transformations:", type(transformations))
if isinstance(transformations, dict):
    print("Clés du dictionnaire:", list(transformations.keys())[:3])
    sample_key = list(transformations.keys())[0]
    print(f"Structure pour la tâche {sample_key}:", type(transformations[sample_key]))
    if isinstance(transformations[sample_key], dict):
        print("Clés de la tâche:", list(transformations[sample_key].keys()))

# Fonction pour extraire les résumés
def extract_summaries(transformations):
    """Extrait les résumés de transformations de la structure de données"""
    summaries = {}
    
    if isinstance(transformations, dict):
        # Structure: {task_id: task_data}
        for task_id, task_data in transformations.items():
            if isinstance(task_data, dict) and "summary" in task_data:
                summaries[task_id] = task_data["summary"]
            elif isinstance(task_data, list):
                # Ancien format: liste de transformations
                # Créer un résumé agrégé
                summary = {}
                for trans in task_data:
                    for t_name, t_value in trans.get("transformations", {}).items():
                        if isinstance(t_value, bool) and t_value:
                            summary[t_name] = summary.get(t_name, 0) + 1
                
                # Convertir en fréquences
                total = len(task_data)
                for t_name in summary:
                    summary[t_name] = summary[t_name] / total
                
                summaries[task_id] = summary
    
    return summaries

# Extrait les résumés
summaries = extract_summaries(transformations)
print(f"Nombre de tâches avec résumés: {len(summaries)}")

# Compte la fréquence des transformations
trans_counter = Counter()

for task_id, summary in summaries.items():
    for trans, freq in summary.items():
        if freq > 0.5:  # Seuil de pertinence
            trans_counter[trans] += 1

# Crée un catalogue ordonné
catalog = []
total_tasks = len(summaries)

for trans, count in trans_counter.most_common():
    # Trouver une tâche exemple contenant cette transformation
    example_task = None
    for task_id, summary in summaries.items():
        if summary.get(trans, 0) > 0.5:
            example_task = task_id
            break
    
    catalog.append({
        "transformation": trans,
        "frequency": count / total_tasks,
        "example_task": example_task
    })

# Sauvegarde en CSV et JSON
df = pd.DataFrame(catalog)
df.to_csv("../data/transformation_catalog.csv", index=False)
with open("../data/transformation_catalog.json", "w") as f:
    json.dump(catalog, f, indent=2)

print(f"Catalogue créé avec {len(catalog)} transformations principales")
print("Transformations les plus courantes:")
print(df.head(10))

Type de l'objet transformations: <class 'dict'>
Clés du dictionnaire: ['00576224', '007bbfb7', '009d5c81']
Structure pour la tâche 00576224: <class 'list'>
Nombre de tâches avec résumés: 1000
Catalogue créé avec 14 transformations principales
Transformations les plus courantes:
               transformation  frequency example_task
0    shape_based_recoloration      0.586     009d5c81
1                recoloration      0.526     009d5c81
2                     padding      0.044     00576224
3                 translation      0.044     00576224
4    pattern_vertical_stripes      0.033     00576224
5  pattern_horizontal_stripes      0.032     00576224
6                rotation_180      0.003     3c9b0459
7                  repetition      0.003     a416b8f3
8               tiled_subgrid      0.003     a416b8f3
9      pattern_uniform_blocks      0.002     28e73c20
